In [30]:
from lxml import etree

NAMESPACES = {"tei": "http://www.tei-c.org/ns/1.0"}

In [29]:
trachiniae = etree.parse("dracor_xml/tlg0011.tlg001.dracor.xml")

In [32]:
speakers = trachiniae.xpath(".//tei:listPerson/person/@id", namespaces=NAMESPACES)